In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('Db-IMDB.db')

# <img src= "db_schema.jpeg">

In [3]:
m=pd.read_sql_query('SELECT * FROM movie',conn)
m.head(4)

,index,MID,title,year,rating,num_votes
0,0,tt2388771,Mowgli,2018,6.6,21967
1,1,tt5164214,Ocean's Eight,2018,6.2,110861
2,2,tt1365519,Tomb Raider,2018,6.4,142585
3,3,tt0848228,The Avengers,2012,8.1,1137529


# Question 1


### List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year

In [43]:
director_table=pd.read_sql_query('SELECT m.title as movie_name,m.year as year, p.name as director_name \
                                 from movie as m left join m_director d on m.mid=d.mid left join person p on d.pid=p.pid \
                                 left join m_genre mg on mg.mid=m.mid left join genre g on mg.gid=g.gid \
                                 where (m.year%4==0 and m.year%100!=0) or (m.year%400==0) and g.name like "%comedy%"',conn)

In [44]:
director_table.tail(3)

,movie_name,year,director_name
1314,Ram Raaj,2008,Kodi Ramakrishna
1315,Goa Dalli CID 999,1968,Dhorairaj Bhagavan
1316,Raja Aur Rangeeli,1996,K.S. Prakash Rao


# Question 2

### 'List the names of all the actors who played in the movie 'Anand' (1971)'

In [45]:
actors_in_anand_movie = pd.read_sql_query( 'SELECT NAME FROM Person WHERE PID IN(SELECT TRIM(PID) \
                                          FROM M_cast where MID IN (SELECT MID FROM Movie WHERE title= "Anand" and year =1971 ))',conn)

In [47]:
actors_in_anand_movie

,Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Sumita Sanyal
3,Ramesh Deo
4,Seema Deo
5,Asit Kumar Sen
6,Dev Kishan
7,Atam Prakash
8,Lalita Kumari
9,Savita


# Question 3

### List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [51]:
actor_id=pd.read_sql_query('SELECT name FROM person WHERE pid IN(SELECT trim(pid) FROM M_cast WHERE MID IN\
                           (SELECT mid FROM movie where year <= 1970 or year >=1990))',conn)

actor_id.head(3)

,Name
0,Christian Bale
1,Cate Blanchett
2,Benedict Cumberbatch


# Question 4

### List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [73]:
directors_done_movies_more_than_10=pd.read_sql_query('SELECT p.name, count(name) name_count from movie as m left \
                                                     join m_director d on m.mid=d.mid left join person p on d.pid=p.pid \
                                                     group by p.name  having name_count>10 order by name_count desc',conn)
directors_done_movies_more_than_10.head(3)

,Name,name_count
0,David Dhawan,39
1,David Dhawan,39
2,Mahesh Bhatt,36


# Question 5

### 5.a. For each year, count the number of movies in that year that had only female actors.

In [6]:
movies_in_a_year_with_only_female=pd.read_sql_query('select max(trim(m.year)) as year, mc.mid,count(distinct m.mid) as Movie_count,count(mc.pid) Female_cnt,count(case when p.gender="Male" then 1 end) as male_cnt from m_cast mc join person p on p.pid = trim(mc.pid) join movie m on m.mid = mc.mid group by mc.mid, m.year having male_cnt = 0',conn)

In [7]:
movies_in_a_year_with_only_female

,year,MID,Movie_count,Female_cnt,male_cnt
0,1999,tt0272001,1,11,0
1,2000,tt0354922,1,11,0
2,1939,tt0375882,1,2,0
3,2018,tt8338754,1,1,0
4,I 2018,tt8458202,1,2,0


### 5.b. Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [8]:
q5_b=pd.read_sql_query('select m1.year, count(m1.mid) as t_Movie_count, m.Movie_count as f_Movie_count , \
                       ROUND(m.Movie_count*100/count(m1.mid),3) as Percentage from movie m1 join \
                       (select max(trim(m.year)) as year, mc.mid,count(distinct m.mid) as Movie_count,\
                        count(mc.pid) F_count,count(case when p.gender="Male" then 1 end) as male_cnt \
                        from m_cast mc join person p on p.pid = trim(mc.pid) join movie m on m.mid = mc.mid \
                        group by mc.mid, m.year having male_cnt = 0) as m on m1.year = m.year group by  m1.year',conn)

In [9]:
q5_b

,year,t_Movie_count,f_Movie_count,Percentage
0,1939,2,1,50.0
1,1999,66,1,1.0
2,2000,64,1,1.0
3,2018,93,1,1.0
4,I 2018,10,1,10.0


# Question 6

### Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [10]:
q6=pd.read_sql_query('select m.title,count(distinct mc.pid) as num_actors from movie m join m_cast mc on mc.mid = m.mid \
                     group by m.mid order by num_actors desc',conn)
q6.head(3)

,title,num_actors
0,Ocean's Eight,238
1,Apaharan,233
2,Gold,215


# Question 7

### A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [11]:
q7=pd.read_sql_query('SELECT  (CAST( (year/10) as int) *10) as decade, count(year) as Total_number_of_movie FROM MOVIE\
                  WHERE decade!=0 group by decade order by Total_number_of_movie desc limit 1',conn)
q7.head(3)

,decade,Total_number_of_movie
0,2010,1018


# Question 8

### Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies).

In [13]:
#https://www.coursehero.com/file/p7mfaba/15-Find-the-actors-who-were-never-unemployed-for-more-than-3-years-at-a-stretch/
actors_employed_3_consecutive_years=pd.read_sql_query('select pid, name from person where pid not in (select distinct(PID)\
                                                      from M_Cast as C1 natural join Movie as \
                                                        M1 where exists\
                                                        (select MID from M_Cast as C2 natural join Movie as M2 \
                                                         where C1.PID = C2.PID and (M2.year - 3) > M1.year and not exists\
                                                         (select MID from M_Cast as C3 natural join Movie as M3 \
                                                    where C1.PID = C3.PID and M1.year < M3.year and M3.year < M2.year)))',conn)
actors_employed_3_consecutive_years.head(3)

,PID,Name
0,nm0000288,Christian Bale
1,nm0000949,Cate Blanchett
2,nm1212722,Benedict Cumberbatch


# Question 9

### Find all the actors that made more movies with Yash Chopra than any other director.

In [14]:
q9=pd.read_sql_query('select p2.pid, p2.name, count(m1.mid) as No_Movies_with_Yash from person p2  \
                join m_cast mc1 on trim(mc1.pid) = p2.pid \
                join movie m1 on m1.mid = mc1.mid \
                join m_director md1 on md1.mid = m1.mid\
                join person p3 on p3.pid = md1.pid where p3.name = "Yash Chopra" group by p2.pid having count(m1.mid) > \
                (select count(m.mid) from person  p \
                    join m_cast mc on trim(mc.pid) = p.pid \
                    join movie m on m.mid = mc.mid \
                    join m_director md on m.mid = md.mid \
                    join person p1 on trim(md.pid) = p1.pid where p2.pid = p.pid and p1.name != "Yash Chopra" group by p1.pid)',conn)

In [15]:
q9.head(3)

,PID,Name,No_Movies_with_Yash
0,nm0000821,Amitabh Bachchan,6
1,nm0004434,Shashi Kapoor,14
2,nm0004435,Rajesh Khanna,3


# Question 10

### The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [18]:
q10=pd.read_sql_query('select distinct p.pid, name from person p \
join m_cast mc on p.pid = trim(mc.pid) where name != " Shah Rukh Khan" and mid in \
(select mc.mid from m_cast mc where trim(mc.pid) in \
(select p.pid from person p join m_cast mc on p.pid = trim(mc.pid) where p.name != " Shah Rukh Khan" and mid in \
(select mc.mid from person p join m_cast mc on trim(mc.pid)=p.pid where p.name = " Shah Rukh Khan" and p.pid not in \
(select p.pid from person p join m_cast mc on p.pid = trim(mc.pid) where p.name != " Shah Rukh Khan" and mid in \
(select mc.mid from person p join m_cast mc on p.pid=trim(mc.pid) where p.name = " Shah Rukh Khan")))))',conn)

In [19]:
q10

,PID,Name
0,nm2539953,Alicia Vikander
1,nm0922035,Dominic West
2,nm0324658,Walton Goggins
3,nm0943079,Daniel Wu
4,nm0000218,Kristin Scott Thomas
...,...,...
28437,nm1506519,Minoo Mehtab
28438,nm2371614,Hayley Cleghorn
28439,nm2675737,Nirvasha Jithoo
28440,nm2370589,Kamal Maharshi
